In [1]:
# Problem 1
import numpy as np
from scipy import linalg as la
def is_drazin(A, k , AD):
    a = np.allclose(np.dot(AD, A), np.dot(A,AD))
    b = np.allclose(np.dot(np.linalg.matrix_power(A, k+1), AD), np.linalg.matrix_power(A, k))
    c = np.allclose(np.dot(np.dot(AD, A),AD), AD)
    d = [a, b, c]
    if all(d) is True:
        return True
    else:
        return False

A = np.array([[1,3,0,0],[0,1,3,0],[0,0,1,3],[0,0,0,0]])
AD = np.array([[1,-3,9,81],[0,1,-3,-18],[0,0,1,3],[0,0,0,0]])
print(is_drazin(A, 1, AD))
B = np.array([[1,1,3],[5,2,6],[-2,-1,-3]])
BD = np.zeros((3,3))
print(is_drazin(B, 3, BD))

True
True


In [2]:
# Problem 2
def drazin(A, tol = 1e-5):
    n, n = np.shape(A)
    f = lambda x: abs(x) > tol
    Q1, S, k1 = la.schur(A, sort=f)
    f = lambda x: abs(x) <= tol
    Q2, T, k2 = la.schur(A, sort=f)
    U = np.concatenate((S[:,:k1],T[:,:n-k1]), axis=1)
    U_inv = la.inv(U)
    V = np.dot(np.dot(U_inv, A), U)
    Z = np.zeros((n,n), dtype = float)
    if k1 != 0:
        M_inv=la.inv(V[:k1,:k1])
        Z[:k1,:k1] = M_inv
    return np.dot(np.dot(U, Z), U_inv)

print(drazin(A))
print(drazin(B))

[[  1.  -3.   9.  81.]
 [  0.   1.  -3. -18.]
 [  0.   0.   1.   3.]
 [  0.   0.   0.   0.]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


In [3]:
# Problem 3
def ef_res(A):
    n,n = np.shape(A)
    # Calculate Laplacian
    diag = np.sum(A,axis=1)
    D = np.diag(diag)
    L = D - A
    I = np.identity(n)
    R = np.zeros((n, n))
    for j in range(n):
        L_tilde = np.copy(L)
        L_tilde[j, :] = I[j, :]
        R[:, j] = np.diag(drazin(L_tilde))
        R[j, j] = 0
    return R

A = np.array([[0,0,1,0],[0,0,0,1],[1,0,0,1],[0,1,1,0]])
print(ef_res(A))
C = np.array([[0,1,1],[1,0,1],[1,1,0]])
print(ef_res(C))

[[ 0.  3.  1.  2.]
 [ 3.  0.  2.  1.]
 [ 1.  2.  0.  1.]
 [ 2.  1.  1.  0.]]
[[ 0.          0.66666667  0.66666667]
 [ 0.66666667  0.          0.66666667]
 [ 0.66666667  0.66666667  0.        ]]


In [6]:
# Problem 4
import pandas as pd

class LinkPredictor():
    def __init__(self, name = 'social_network.csv'):
        
        data = pd.read_csv(name, header = None, index_col=False).astype(str)
        data = data.values
        m = np.shape(data)[0]
        ppl, index = np.unique(data, return_inverse=True)
        self.names = ppl
        index = index.reshape(m, 2)
        
        adj = np.zeros((len(ppl) , len(ppl)))
        for i in range(m):
            adj[index[i, 0], index[i, 1]] = 1
            adj[index[i, 1], index[i, 0]] = 1
            
        self.adjacency = adj  
        self.resistance = ef_res(adj)
    
    def predict_link(self, node = None):
        R = self.resistance
        A = self.adjacency
        N = self.names
        R = R * [A==0]
        R[R==0] = np.inf
        
        if node is None:
            minres = np.min(R)
            nodes = np.where(R == minres)
            return N[nodes[0]], N[nodes[1]]
        elif np.any(N[N == node]):
            node1 = np.where(N == node)[0][0]
            minres = np.min(R[0][node1,:])
            node2 = np.where(R[0][node1,:] == minres)
            return N[node2[0][0]]
        else:
            raise ValueError("{} is not in network".format(node))
        
    def add_link(self, name1, name2):
        R = self.resistance
        A = self.adjacency
        N = self.names
        if np.any(N[N == name1]) and np.any(N[N == name2]):
            index1 = np.where(N==name1)[0][0]
            index2 = np.where(N==name2)[0][0]
            A[index1,index2] = A[index1,index2]+1
            A[index2,index1] = A[index2,index1]+1
            R = ef_res(A)
            self.adjacency = A
            self.resistance = R
        else:
            raise ValueError("{} or {} is not in network".format(name1, name2))

In [7]:
data = LinkPredictor()
print(data.predict_link())
print(data.predict_link('Melanie'))
print(data.predict_link('Alan'))
data.add_link('Alan', 'Sonia')
print(data.predict_link('Alan'))
data.add_link('Alan', 'Piers')
print(data.predict_link('Alan'))

(array(['Abigail'], dtype=object), array(['Oliver'], dtype=object))
Carol
Sonia
Piers
Abigail
